In [1]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix
from visualization import load_depth, load_stencil, save_pointcloud_csv, bbox_from_string
import progressbar
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle
import json
import time
from shutil import copyfile
from functools import lru_cache

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file
visualization.use_cache = False

conn = visualization.get_connection_pooled()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = r'D:\output-datasets\offroad-8'

# for some datasets we don't need entities, which produces lot smaller json files
include_entities = False
# for some datasets, we want each camera data to be in separate folder
directory_per_camera = True
camera_names = {}
# sometimes buffers are not synced and don't correspond to current camera. 
# they can be recognized by having same data as previous record
discard_invalid_buffers = True

In [3]:
#run_id = 19
#run_id = 3677
run_id = 4149

cur.execute("""SELECT snapshot_id, imagepath, cam_near_clip, camera_fov, width, height, timestamp, timeofday, width, height, \
      ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as camera_relative_rotation, \
      ARRAY[st_x(camera_relative_position), st_y(camera_relative_position), st_z(camera_relative_position)] as camera_relative_position, \
      ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos, \
      ARRAY[st_x(camera_rot), st_y(camera_rot), st_z(camera_rot)] as camera_rot, \
      ARRAY[st_x(current_target), st_y(current_target), st_z(current_target)] as current_target, \
      currentweather \
      FROM snapshots \
      WHERE run_id = {}
      ORDER BY timestamp ASC \
    """.format(run_id))

results = []
for row in cur:
    res = dict(row)
    if res['camera_fov'] == 0 and res['cam_near_clip'] == 0:
        continue  # somehow malformed data, skipping them
    res['camera_rot'] = np.array(res['camera_rot'])
    res['camera_pos'] = np.array(res['camera_pos'])
    res['camera_relative_rotation'] = np.array(res['camera_relative_rotation'])
    res['camera_relative_position'] = np.array(res['camera_relative_position'])
    res['current_target'] = np.array(res['current_target'])
    res['view_matrix'] = construct_view_matrix(res['camera_pos'], res['camera_rot'])
    res['proj_matrix'] = construct_proj_matrix(res['height'], res['width'], res['camera_fov'], res['cam_near_clip'])
    results.append(res)

print('There are {} snapshots'.format(len(results)))

There are 3840 snapshots


In [4]:
def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])

def get_main_image_name(cameras):
    for cam in cameras:
        # this is the main camera
        if np.array_equal(cam['camera_relative_rotation'], [0, 0, 0]):
            return cam['imagepath']
    raise Exception('no main image')

def load_entities_data(snapshot_id):
    conn = visualization.get_connection_pooled()
    cur = conn.cursor()
    
    # start = time.time()

    cur.execute("""SELECT bbox, \
        ARRAY[st_x(pos), st_y(pos), st_z(pos)] as pos, \
        ARRAY[st_x(rot), st_y(rot), st_z(rot)] as rot, \
        ARRAY[st_xmin(bbox3d), st_xmax(bbox3d), st_ymin(bbox3d), st_ymax(bbox3d), st_zmin(bbox3d), st_zmax(bbox3d)] as bbox3d, \
         type, class, handle, snapshot_id \
        FROM detections \
        WHERE snapshot_id = '{}' \
        """.format(snapshot_id))

    # end = time.time()
    # print('time to load from db', end - start)
    # start = time.time()

    # print(size)
    results = []
    for row in cur:
        res = dict(row)
        res['model_sizes'] = np.array(res['bbox3d'])
        res['bbox'] = bbox_from_string(res['bbox'])
        res['pos'] = np.array(res['pos'])
        res['rot'] = np.array(res['rot'])
        results.append(res)

    # end = time.time()
    # print('time to convert arrays to numpy', end - start)
    # start = time.time()

    return results


def convert_rgb(in_directory, out_directory, out_name, name, out_format):
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    # print(outfile)
    if os.path.exists(outfile):
        return

    try:
        infile = os.path.join(in_directory, name)
        im = Image.open(infile)
        im = im.convert(mode="RGB")
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return


def convert_depth(in_directory, out_directory, out_name, name, out_format):
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    # print(outfile)
    if os.path.exists(outfile):
        return

    try:
        infile = os.path.join(in_directory, name)
        depth = load_depth(name)
        # print('depth min before: ', np.min(depth))
        # print('depth max before: ', np.max(depth))
        depth = depth * np.iinfo(np.uint16).max
        # print('depth min after: ', np.min(depth))
        # print('depth max after: ', np.max(depth))
        im = Image.fromarray(depth.astype(np.int32), mode="I")
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return

def convert_stencil(in_directory, out_directory, out_name, name, out_format):
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    # print(outfile)
    if os.path.exists(outfile):
        return

    try:
        infile = os.path.join(in_directory, name)
        stencil = load_stencil(name)
        im = Image.fromarray(stencil.astype(np.uint8), mode="L")
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return

def get_out_directory(out_directory, res, directory_per_camera):
    if directory_per_camera:
        # dictionary of camera string representation and indices
        global camera_names
        camera_directory_name = 'camera_{}__{}'.format(
            '_'.join(['{:0.2f}'.format(i) for i in res['camera_relative_position']]), 
            '_'.join(['{:0.2f}'.format(i) for i in res['camera_relative_rotation']]), 
        )
        if camera_directory_name not in camera_names:
            camera_names[camera_directory_name] = str(len(camera_names))
        return os.path.join(out_directory, camera_names[camera_directory_name])
    else:
        return out_directory

@lru_cache(maxsize=32)
def get_first_record_timestamp_in_run(run_id):
    conn = visualization.get_connection_pooled()
    cur = conn.cursor()
    cur.execute("""SELECT min(timestamp) as timestamp \
        FROM snapshots \
        WHERE run_id = {} \
        LIMIT 1 \
        """.format(run_id))
    return cur.fetchone()['timestamp']
    
def is_first_record_in_run(res, run_id):
    first_timestamp = get_first_record_timestamp_in_run(run_id)
    return first_timestamp == res['timestamp']
    
def get_previous_record(res):
    conn = visualization.get_connection_pooled()
    cur = conn.cursor()
    cur.execute("""SELECT imagepath, snapshot_id, scene_id \
        FROM snapshots \
        WHERE timestamp < '{}' and run_id = (SELECT run_id from snapshots WHERE snapshot_id = {}) \
        ORDER BY timestamp DESC \
        LIMIT 1 \
        """.format(res['timestamp'], res['snapshot_id']))
    # this should select previous record independently on primary key, without problems with race conditions by persisting in other threads
    # and belonging into the same run
    results = []
    for row in cur:
        res = dict(row)
        results.append(res)
    if len(results) == 0:
        print('no previous record for snapshot_id {}'.format(res['snapshot_id']))
    return results[0]['imagepath']

def are_buffers_same_as_previous(in_directory, res):
    name = res['imagepath']
    depth = load_depth(name)
    stencil = load_stencil(name)
    prev_name = get_previous_record(res)
    prev_depth = load_depth(prev_name)
    prev_stencil = load_stencil(prev_name)
    return (depth == prev_depth).all() or (stencil == prev_stencil).all()

def try_dump_snapshot_to_dataset(in_directory, out_directory, res, run_id):
    try:
        dump_snapshot_to_dataset(in_directory, out_directory, res, run_id)
    except Exception as e:
        print(e)
        pass

def dump_snapshot_to_dataset(in_directory, out_directory, res, run_id):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    start = time.time()
    
    if not os.path.exists(out_directory):
        os.makedirs(out_directory)

    if discard_invalid_buffers and (not is_first_record_in_run(res, run_id)) and \
        are_buffers_same_as_previous(in_directory, res):
        print('skipping record wih invalid buffers in snapshot {}, filename {} and camera {}'
              .format(res['snapshot_id'], res['imagepath'], res['camera_relative_position'].tolist()))
        return
    
    name = res['imagepath']
#    convert_rgb(in_directory, out_directory, get_base_name(name), get_base_name(name)+'.tiff', 'png')
    convert_rgb(in_directory, out_directory, get_base_name(name), get_base_name(name)+'.tiff', 'jpg')
    convert_depth(in_directory, out_directory, get_base_name(name)+'-depth', get_base_name(name), 'png')
    convert_stencil(in_directory, out_directory, get_base_name(name)+'-stencil', get_base_name(name), 'png')
    # end = time.time()
    # print('time to generate images', end - start)
    # start = time.time()
    
    outfile = os.path.join(out_directory, '{}.json'.format(res['imagepath']))
    if os.path.exists(outfile):
        return

    if include_entities:
        data = load_entities_data(res['snapshot_id'])

        # end = time.time()
        # print('loading entities data', end - start)
        # start = time.time()
    
        json_entities_data = []
        for i in data:
            json_entity = {
                'model_sizes': i['model_sizes'].tolist(),
                'bbox': i['bbox'].tolist(),
                'pos': i['pos'].tolist(),
                'rot': i['rot'].tolist(),
                'class': i['class'],
                'handle': i['handle'],
                'type': i['type'],
            }
            json_entities_data.append(json_entity)

    # end = time.time()
    # print('dumping entities to json', end - start)
    # start = time.time()

    json_data = {
        'imagepath': res['imagepath'],
        'timestamp': res['timestamp'].strftime("%Y-%m-%d %H:%M:%S"),
        'timeofday': res['timeofday'].strftime("%H:%M:%S"),
        'currentweather': res['currentweather'],
        'width': res['width'],
        'height': res['height'],
        'snapshot_id': res['snapshot_id'],
        'camera_rot': res['camera_rot'].tolist(),
        'camera_pos': res['camera_pos'].tolist(),
        'camera_fov': res['camera_fov'],
        'camera_relative_rotation': res['camera_relative_rotation'].tolist(),
        'camera_relative_position': res['camera_relative_position'].tolist(),
        'current_target': res['current_target'].tolist(),
        'view_matrix': res['view_matrix'].tolist(),
        'proj_matrix': res['proj_matrix'].tolist(),
    }
    
    if include_entities:
        json_data['entities']: json_entities_data

    # end = time.time()
    # print('dumping base object to json', end - start)
    # start = time.time()

    with open(outfile, 'w') as f:
        json.dump(json_data, f)
    
    # end = time.time()
    # print('persisting json to file', end - start)
    

### dumping all data

In [5]:
workers = 8

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(results)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(try_dump_snapshot_to_dataset)(in_directory, get_out_directory(out_directory, i, directory_per_camera), i, run_id) for i in results)
print('done')

  5% 223 |###                                                       |   4.4 B/sC:\ProgramData\Anaconda3\lib\site-packages\tifffile\tifffile.py:2642: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
  5% 225 |###                                                       |   4.4 B/sC:\ProgramData\Anaconda3\lib\site-packages\tifffile\tifffile.py:2577: UserWarning: unpack: string size must be a multiple of element size
  warnings.warn("unpack: %s" % e)
  8% 336 |#####                                                     |   4.4 B/s

skipping record wih invalid buffers in snapshot 469934, filename 2018-08-13--11-34-41--826 and camera [0.480000019073486, 1.5, 1.07649993896484]


  8% 338 |#####                                                     |   4.4 B/s

skipping record wih invalid buffers in snapshot 469935, filename 2018-08-13--11-34-42--114 and camera [-0.0599999986588955, 17.5, 25.0764999389648]


  8% 339 |#####                                                     |   4.4 B/s

...

skipping record wih invalid buffers in snapshot 473128, filename 2018-08-13--11-52-44--704 and camera [-0.0599999986588955, 1.5, 1.07649993896484]
skipping record wih invalid buffers in snapshot 473129, filename 2018-08-13--11-52-45--005 and camera [15.9399995803833, 17.5, 6.07649993896484]
skipping record wih invalid buffers in snapshot 473130, filename 2018-08-13--11-52-45--278 and camera [-0.0599999986588955, 33.5, 6.07649993896484]
skipping record wih invalid buffers in snapshot 473133, filename 2018-08-13--11-52-46--207 and camera [-0.0599999986588955, 17.5, 25.0764999389648]


 92% 3535 |####################################################     |   4.1 B/s

skipping record wih invalid buffers in snapshot 473131, filename 2018-08-13--11-52-45--583 and camera [-16.0599994659424, 17.5, 6.07649993896484]
skipping record wih invalid buffers in snapshot 473132, filename 2018-08-13--11-52-45--906 and camera [0.480000019073486, 1.5, 1.07649993896484]


100% 3840 |#########################################################|   4.1 B/s

done


### copying to linux into datagrid dir

In [ ]:
def copy_file(in_path, out_path):
    if os.path.exists(out_path):
        return
    copyfile(in_path, out_path)

def copy_files(in_dir, out_dir, name):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    rgb_name = name+'.jpg'
    json_name = name+'.json'
    depth_name = name+'-depth.png'
    stencil_name = name+'-stencil.png'
    
    copyfile(os.path.join(in_dir, rgb_name), os.path.join(out_dir, rgb_name))
    copyfile(os.path.join(in_dir, json_name), os.path.join(out_dir, json_name))
    copyfile(os.path.join(in_dir, depth_name), os.path.join(out_dir, depth_name))
    copyfile(os.path.join(in_dir, stencil_name), os.path.join(out_dir, stencil_name))

workers = 8

windows_in_directory = r'D:\output-datasets\offroad-6'
linux_out_directory = r'Z:\offroad-6'

if not os.path.exists(linux_out_directory):
    os.makedirs(linux_out_directory)

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, max_value=len(results)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(
    delayed(copy_files)(windows_in_directory, linux_out_directory, i['imagepath']) for i in results)


  8% 812 |#####                                                     |   2.0 B/s

### copying original tiffs to linux into datagrid dir, only one run from database

In [42]:
def copy_file(in_path, out_path):
    if os.path.exists(out_path):
        return
    copyfile(in_path, out_path)

def copy_files(in_dir, out_dir, name):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    rgb_name = name+'.tiff'
    depth_name = name+'-depth.tiff'
    stencil_name = name+'-stencil.tiff'
    
    copyfile(os.path.join(in_dir, rgb_name), os.path.join(out_dir, rgb_name))
    copyfile(os.path.join(in_dir, depth_name), os.path.join(out_dir, depth_name))
    copyfile(os.path.join(in_dir, stencil_name), os.path.join(out_dir, stencil_name))

workers = 8

out_directory = r'Z:\offroad-4-orig'
widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, max_value=len(results)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(
    delayed(copy_files)(in_directory, out_directory, i['imagepath']) for i in results)


  0% 42 |                                                           |   0.7 B/s

KeyboardInterrupt: 

### copying videos into datagrid

### dumping one snapshot, for testing

In [ ]:
dump_snapshot_to_dataset(in_directory, out_directory, results[0])


In [ ]:
im = np.array(Image.open(os.path.join(out_directory, results[0]['imagepath']+'-depth.png')))
print(np.min(im))
print(np.max(im))


In [ ]:
results[0]['snapshot_id']

In [64]:
res = results[0]
res

{'cam_near_clip': 0.15,
 'camera_fov': 90.0,
 'camera_pos': array([ 2524.83105469,  3334.53613281,    53.38750458]),
 'camera_relative_position': array([-0.06      ,  1.5       ,  1.07649994]),
 'camera_relative_rotation': array([ 0.,  0.,  0.]),
 'camera_rot': array([-1.26793635, -4.02545595, -0.25146627]),
 'current_target': array([2586.0, 3490.0, None], dtype=object),
 'currentweather': 'Clear',
 'height': 1052,
 'imagepath': '2018-08-13--11-15-01--499',
 'proj_matrix': array([[  5.49634274e-01,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   1.49945045e-05,
           1.50002249e-01],
        [  0.00000000e+00,   0.00000000e+00,  -1.00000000e+00,
           0.00000000e+00]]),
 'snapshot_id': 466517,
 'timeofday': datetime.time(12, 5, 35),
 'timestamp': datetime.datetime(2018, 8, 13, 11, 15, 1, 499713, tzinfo=psycopg2.tz.FixedOffse